In [1]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder
from cosapp.drivers import NonLinearSolver, RunOnce, MonteCarlo, RunSingleCase
from cosapp.utils.distributions import Normal
import sys
sys.path.append('./model')
from Earth import Earth
import json
import numpy as np

earth = Earth("earth")

l = 2 #Rocket's length on the plot
dt = 0.1 #Time-step


from cosapp.utils.distributions import Normal
from cosapp.drivers import MonteCarlo
from cosapp.recorders import DataFrameRecorder

In [2]:

#Initial conditions and constants

rocketNum = 2622
l = 0.855
angz = -np.deg2rad(85)
expCoef = False #True for experimental drag coefficient, false for analytical (possibly innacurate)
Wind = False #True for wind effects, false for no wind
Lift = False #True for lift effects (trajecto does not consider them), false for no lift

init = {
    'Traj.r' : np.array([-(l/2)*np.sin(angz), 0., (l/2)*np.cos(angz)]),
    'Para.DynPar.r1' : np.array([0., 0., l/2]),
    'Para.DynPar.r2' : np.array([0., 0., l/2]),
    'Rocket.CG' : l/2,
    
    'Rocket.Mass.m' : 2.0783,
    'Rocket.Mass.m0' : 2.0783,
    'Rocket.Mass.Dm' : 0.076,
    'Rocket.Mass.lastEngineTime' : 1.,
    'Rocket.Mass.I0_geom' : 1*np.array([1., 100., 100.]),

    'Rocket.Kin.v' : np.array([0,0,0]),
    'Rocket.Kin.av' : np.zeros(3),
    'Para.DynPar.v1' : np.array([0,0,0]),
    'Para.DynPar.v2' : np.array([0,0,0]),
    'Traj.ParaDepStatus': False,
    'Rocket.Kin.ar' : 'initrot',

    'Rocket.Aero.Coefs.ln' : 0.159,
    'Rocket.Aero.Coefs.d' : 0.08,
    'Rocket.Aero.Coefs.NFins' : 4,
    'Rocket.Aero.Coefs.s' : 0.13,
    'Rocket.Aero.Coefs.Xt' : 0.09,
    'Rocket.Aero.Coefs.Cr' : 0.145,
    'Rocket.Aero.Coefs.Ct' : 0.09,
    'Rocket.Aero.Coefs.tf' : 0.002,
    'Rocket.Aero.Coefs.delta' : 0.,

    'Rocket.Aero.Coefs.TypeCd' : expCoef,
    'Wind.wind_on' : Wind,
    'Rocket.Aero.Aeroforces.isLift': Lift,
    'Rocket.Aero.Coefs.Cd_exp': 0.6,

    'Traj.parachute_deploy_method' : 0,
    'Traj.parachute_deploy_timer' : 0.,

}

In [3]:
def run_analysis(syst, draws=10, linear=True):
    syst.drivers.clear()  # Remove all drivers on the System

    runonce = syst.add_driver(RunOnce("runonce"))
    driver = syst.add_driver(RungeKutta(order=4, dt=dt))
    # driver.add_recorder(DataFrameRecorder(includes=['Rocket.Kin.ar', 'Traj.r', 'Rocket.Kin.v', 'initrot', 'Rocket.Thrust.inclinaison']), period=0.1)
    driver.add_child(NonLinearSolver('solver', factor=1.0))
    driver.time_interval = (0, 70)

    # Define a simulation scenario
    driver.set_scenario(
        init = init,
        stop = 'Para.DynPar.r2[2] < -1'
    )

    syst.run_drivers()

    print("MONTECARLO")
    # Montecarlo
    syst.drivers.clear()
    montecarlo = syst.add_driver(MonteCarlo('mc'))
    montecarlo.add_recorder(DataFrameRecorder(includes=['Para.DynPar.r2', 'initrot', 'Para.DynPar.v2', 'Rocket.Thrust.inclinaison']))
    montecarlo.add_child(driver)
    montecarlo.draws = draws
    # montecarlo.linear = linear

    # parameters for uncertainties in the data
    pitch_attr = syst.inwards.get_details('pitch_init')
    yaw_attr = syst.inwards.get_details('yaw_init')
    bang_attr = syst.Rocket.Thrust.inwards.get_details('inclinaison')
    
    # Set the distribution around the current value
    pitch_attr.distribution = Normal(best=.2, worst=-.2)
    yaw_attr.distribution = Normal(best=.2, worst=-.2)
    bang_attr.distribution = Normal(best=.4, worst=-0.4)

    montecarlo.add_random_variable('pitch_init')
    montecarlo.add_random_variable('yaw_init')
    montecarlo.add_random_variable('Rocket.Thrust.inclinaison')

    # Computation
    syst.run_drivers()

    return montecarlo.recorder.export_data()

In [4]:

results = run_analysis(earth, draws=100, linear=False)

___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  34.42262886999433 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  164.16817238578332 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
MONTECARLO
___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  34.42262886999433 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  164.16817238578327 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  34.42262886999433 m
Rocket Y Coordinate at Deployment:  0.0 m
Rocket Z Coordinate at Deployment:  164.16817238578332 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  34.42262886999433 m
Rocket Y Coordinate at Deployment:  -4.1706909723223006e-15 m
Rocket Z Coordinate at Deployment:  164.16817238578332 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !
_

In [5]:
results

,Section,Status,Error code,Reference,Para.DynPar.r2,Para.DynPar.v2,Rocket.Thrust.inclinaison,initrot
0,,,0,0,"[43.29292759998309, -5.289899827577889e-15, -0...","[1.215783941984121e-06, -1.5404278934484204e-2...",-1.091401e-17,"[0.0, -1.4707963267948965, -1.091401485662647e..."
1,,,0,1,"[44.97458740139474, -2.7716828680379946, -0.99...","[1.2687107209183277e-06, -1.2534281237544465e-...",1.301559e-01,"[0.0, -1.6009522551996178, 0.13015592840472132]"


In [8]:

with open('result.txt', 'a') as fp:
    fp.write(results.to_csv())

In [7]:
traj = np.asarray(results['Para.DynPar.r2'].tolist())
traj

array([[ 4.32929276e+01,  0.00000000e+00, -1.00000000e+00],
       [ 4.22011819e+01,  1.19279093e-14, -1.00000000e+00]])

In [6]:

import plotly.graph_objs as go
from plotly.subplots import make_subplots

traj = np.asarray(results['Para.DynPar.r2'].tolist())

# Create the figure object
fig = make_subplots(rows=1, cols=1)
fig.layout.title = "Probability"
fig.layout.yaxis.title = 'Y Position'

fig.add_trace(
    go.Scatter(
        x=traj[:,0],
        y=traj[:,1],
        mode = 'markers'
    ),
    row = 1,
    col = 1,
)
fig.get_subplot(1, 1).xaxis.title = "X Position"

fig.show()